#Fine Tuning BERTLM

In [1]:
!pip install transformers

     |████████████████████████████████| 1.5MB 6.4MB/s 
     |████████████████████████████████| 2.9MB 27.0MB/s 
     |████████████████████████████████| 890kB 56.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=68636969ef2e80c776c0b435e1fcbf75c0affabd7d34a26b4bb3816cb9bd219f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import BertTokenizer, BertForMaskedLM
from transformers import LineByLineTextDataset

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
dataset = LineByLineTextDataset(tokenizer=tokenizer,
                                file_path = "/content/drive/MyDrive/data_aug/eng_text.txt",
                                block_size = 128,
                                )

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [5]:

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [6]:
!mkdir output_model

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/output_model",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=500,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [8]:
trainer.train()

Step,Training Loss
500,1.086780
1000,0.797049
1500,0.781471
2000,0.751950
2500,0.769059
3000,0.730843
3500,0.740509
4000,0.751663
4500,0.731309
5000,0.720983


TrainOutput(global_step=39492, training_loss=0.6721904611292913)

In [9]:
trainer.save_model("/content/drive/MyDrive/data_aug")

In [10]:
from transformers import pipeline

In [11]:
fill_mask = pipeline(
    "fill-mask",
    model = "/content/drive/MyDrive/data_aug",
    tokenizer = "bert-base-uncased"
)

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/data_aug and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
fill_mask("A product [MASK] is the marketing copy that explains what a product is and why it’s worth purchasing. ")

[{'score': 0.07122289389371872,
  'sequence': '[CLS] a product label is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SEP]',
  'token': 3830,
  'token_str': 'label'},
 {'score': 0.052535202354192734,
  'sequence': '[CLS] a product register is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SEP]',
  'token': 4236,
  'token_str': 'register'},
 {'score': 0.0298884566873312,
  'sequence': '[CLS] a product list is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SEP]',
  'token': 2862,
  'token_str': 'list'},
 {'score': 0.028533369302749634,
  'sequence': '[CLS] a product licence is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SEP]',
  'token': 11172,
  'token_str': 'licence'},
 {'score': 0.026572827249765396,
  'sequence': '[CLS] a product description is the marketing copy that explains what a product is and why it ’ s worth purchasing. [SE

In [13]:
fill_mask("Software Development Life Cycle is the [MASK] of standard business practices to building software applications.")

[{'score': 0.44959667325019836,
  'sequence': '[CLS] software development life cycle is the application of standard business practices to building software applications. [SEP]',
  'token': 4646,
  'token_str': 'application'},
 {'score': 0.2265416830778122,
  'sequence': '[CLS] software development life cycle is the adaptation of standard business practices to building software applications. [SEP]',
  'token': 6789,
  'token_str': 'adaptation'},
 {'score': 0.11701046675443649,
  'sequence': '[CLS] software development life cycle is the introduction of standard business practices to building software applications. [SEP]',
  'token': 4955,
  'token_str': 'introduction'},
 {'score': 0.06402815133333206,
  'sequence': '[CLS] software development life cycle is the contribution of standard business practices to building software applications. [SEP]',
  'token': 6691,
  'token_str': 'contribution'},
 {'score': 0.013166704215109348,
  'sequence': '[CLS] software development life cycle is the ex